In [1]:
from bandit import Bandits_one
from mab import EpsilonGreedy

# Part 2 - Report

### Exercise 8

In [ ]:
env = Bandits_one()

First we run the Epsilon-greedy MAB algorithm on the environment for 1000 episodes, 20 times.

In [ ]:
# Epsilon-greedy model
agent = EpsilonGreedy()
observation, reward, terminated, truncated, info = env.reset()
for episode in range(1, 1001):
    action = agent.sample()
    observation, reward, terminated, truncated, info = env.step(action)

    agent.update(action, reward, episode)

    if terminated or truncated:
        observation, reward, terminated, truncated, info = env.reset()

Then we run decaying Epsilon-greedy model the same number of times.

In [ ]:
# Decaying Epsilon-greedy model
agent = EpsilonGreedy(alpha=0.5)
observation, reward, terminated, truncated, info = env.reset()
for episode in range(1, 1001):
    action = agent.sample()
    observation, reward, terminated, truncated, info = env.step(action)

    agent.update(action, reward, episode)

    if terminated or truncated:
        observation, reward, terminated, truncated, info = env.reset()